In [ ]:
#Install the dependecies
pip install duckdb


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#import the packages
import json
from datetime import datetime
import duckdb

### Task 1 Data Ingestion

##### I went with flat schema using just one listens table. Since the dataset doesn’t include much user or track metadata, breaking it into multiple normalized tables (like separate user or track tables) would only add extra joins without really helping performance or storage. The analysis tasks are mostly read-heavy and focus on aggregations, so keeping everything in one table made the queries faster and the structure easier to work with.

##### Most of the other columns (release_mbid, recording_mbid, release_group_mbid, artist_mbids, etc.) are  null or empty arrays.
- For this assignment, including those doesn’t add value.

- Hence I skiped them to keep the table lean and focused.

##### To avoid duplicates, I used the following combination as a UNIQUE constraint:
- (user_name, recording_msid, listened_at)

##### For faster querying by date, I added a listened_date column and created an index on it:
- CREATE INDEX idx_listens_listened_date ON listens(listened_date)

In [ ]:
def etl_load_listens(export_file_path, db_path='music_listens.db'):
    con = duckdb.connect(db_path)
    cur = con.cursor()
    #cur.execute('DROP TABLE IF EXISTS listens')

    # Creating table with a unique constraint on columns that identify uniqueness
    cur.execute('''
        CREATE TABLE IF NOT EXISTS listens (
            user_name TEXT,
            artist_name TEXT,
            track_name TEXT,
            release_name TEXT,
            listened_at TIMESTAMP,
            recording_msid TEXT,
            artist_msid TEXT,
            release_msid TEXT,
            listened_date DATE,
            UNIQUE (user_name, recording_msid, listened_at)
        )
    ''')
    
    cur.execute('CREATE INDEX idx_listens_listened_date ON listens(listened_date)')

    with open(export_file_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        line = line.strip()
        if not line:
            continue
        try:
            data = json.loads(line)

            user_name = data.get('user_name')
            listened_at_ts = data.get('listened_at')
            listened_at = datetime.fromtimestamp(listened_at_ts) if listened_at_ts else None
            recording_msid = data.get('recording_msid')


            metadata = data.get('track_metadata', {})
            artist_name = metadata.get('artist_name')
            track_name = metadata.get('track_name')
            release_name = metadata.get('release_name')

            additional_info = metadata.get('additional_info', {})
            artist_msid = additional_info.get('artist_msid')
            release_msid = additional_info.get('release_msid')
            listened_date = listened_at.date() if listened_at else None

            cur.execute('''
                INSERT INTO listens
                (user_name, artist_name, track_name, release_name, listened_at, recording_msid, artist_msid, release_msid, listened_date)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?,?)
                ON CONFLICT DO NOTHING
            ''', (user_name, artist_name, track_name, release_name, listened_at, recording_msid, artist_msid, release_msid, listened_date))

        except (json.JSONDecodeError, KeyError, ValueError) as e:
            # Skip corrupted or incomplete lines
            print(f"Skipping corrupted line: {line}\nError: {e}")

    cur.commit()
    print("ETL load complete.")

    cur.table('listens').show()
    cur.close()

etl_load_listens('data/dataset.txt')

ETL load complete.
┌────────────┬──────────────────────┬────────────────────────────────────┬────────────────────────────────────────────────────────────────┬─────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬───────────────┐
│ user_name  │     artist_name      │             track_name             │                          release_name                          │     listened_at     │            recording_msid            │             artist_msid              │             release_msid             │ listened_date │
│  varchar   │       varchar        │              varchar               │                            varchar                             │      timestamp      │               varchar                │               varchar                │               varchar                │     date      │
├────────────┼──────────────────────┼────────────────────────────────────┼──────────────────────

### Task 2 Data Analysis

#### a) Who are the top 10 users with respect to number of songs listened to?

In [4]:
con = duckdb.connect('music_listens.db')
cur = con.cursor()

cur.sql('''
    SELECT 
        user_name, 
        COUNT(user_name) AS number_of_songs_listened
    FROM listens
    GROUP BY user_name
    ORDER BY number_of_songs_listened DESC
    LIMIT 10
''').show()

┌────────────────┬──────────────────────────┐
│   user_name    │ number_of_songs_listened │
│    varchar     │          int64           │
├────────────────┼──────────────────────────┤
│ hds            │                    46885 │
│ Groschi        │                    14959 │
│ Silent Singer  │                    13005 │
│ phdnk          │                    12861 │
│ 6d6f7274686f6e │                    11544 │
│ reverbel       │                     8398 │
│ Cl�psHydra     │                     8318 │
│ InvincibleAsia │                     7804 │
│ cimualte       │                     7356 │
│ inhji          │                     6349 │
├────────────────┴──────────────────────────┤
│ 10 rows                         2 columns │
└───────────────────────────────────────────┘



#### How many users did listen to some song on the 1st of March 2019?

In [5]:
cur.sql('''
    SELECT   
        COUNT(user_name) AS count
    FROM listens
    WHERE listened_date = '2019-03-01'
        ''').show()

┌───────┐
│ count │
│ int64 │
├───────┤
│  2522 │
└───────┘



#### For every user, what was the first song the user listened to?

In [6]:
cur.sql('''
    WITH first_song AS(
        SELECT 
            user_name, 
            MIN(listened_at) as listened_at 
        FROM listens group by  user_name)

    SELECT 
        DISTINCT f.user_name, 
        l.track_name 
    FROM listens l
    JOIN first_song f ON f.listened_at = l.listened_at AND l.user_name = f.user_name
        ''').show()

┌─────────────────┬───────────────────────────────────────────────┐
│    user_name    │                  track_name                   │
│     varchar     │                    varchar                    │
├─────────────────┼───────────────────────────────────────────────┤
│ toby3d          │ For the Damaged Coda                          │
│ Mexikola        │ Still Clean                                   │
│ Wolfynight      │ Brainless                                     │
│ shogazefan      │ Pange linqua gloriosi, BuxWV 91               │
│ bojarin3        │ Marionette                                    │
│ InvincibleAsia  │ Love Hurts                                    │
│ Zastai          │ Fear                                          │
│ AscendedGravity │ Amoeba                                        │
│ cirfis          │ Crossing the Frame                            │
│ Tinymeat        │ A deep sea meditation experience              │
│    ·            │             ·               

#### b) Top 3 Listening Days per User

In [7]:
cur.sql(''' 
    WITH top_listen AS (
        SELECT
            user_name,
            listened_date AS date,
            COUNT(*) AS number_of_listens,
		    ROW_NUMBER() OVER (
                PARTITION BY user_name
                ORDER BY COUNT(*) DESC
            ) AS rn
        FROM listens
        GROUP BY user_name, listened_date
    )
    SELECT
        user_name,
        number_of_listens,
        date
    FROM top_listen
    WHERE rn <= 3
    ORDER BY user_name, number_of_listens DESC;
        ''').show()

┌─────────────────┬───────────────────┬────────────┐
│    user_name    │ number_of_listens │    date    │
│     varchar     │       int64       │    date    │
├─────────────────┼───────────────────┼────────────┤
│ 6d6f7274686f6e  │               200 │ 2019-01-27 │
│ 6d6f7274686f6e  │               195 │ 2019-01-14 │
│ 6d6f7274686f6e  │               193 │ 2019-01-16 │
│ Adsky_traktor   │               109 │ 2019-01-03 │
│ Adsky_traktor   │                99 │ 2019-01-05 │
│ Adsky_traktor   │                86 │ 2019-01-04 │
│ AllSparks       │               114 │ 2019-01-31 │
│ AllSparks       │                81 │ 2019-01-23 │
│ AllSparks       │                71 │ 2019-01-11 │
│ AlwinHummels    │                 1 │ 2019-02-24 │
│    ·            │                 · │     ·      │
│    ·            │                 · │     ·      │
│    ·            │                 · │     ·      │
│ yellams         │               157 │ 2019-02-15 │
│ zebedeemcdougal │                81 │ 2019-0

#### c) Daily Active Users and Their Percentage of Total Users

In [8]:
cur.sql(''' 
    WITH calendar AS (
        SELECT DISTINCT listened_date FROM listens
    ), 
    rolling_window AS (
        SELECT
            d.listened_date AS date,
            l.user_name
        FROM calendar d
        JOIN listens l
            ON l.listened_date BETWEEN d.listened_date - INTERVAL '6 days' AND d.listened_date
    )
    SELECT
        date,
        COUNT(DISTINCT user_name) AS number_active_users,
        ROUND(COUNT(DISTINCT user_name) * 100.0 / (
            SELECT COUNT(DISTINCT user_name) FROM listens
        ), 2) AS percentage_active_users
    FROM rolling_window
    GROUP BY date
    ORDER BY date
''').show()

┌────────────┬─────────────────────┬─────────────────────────┐
│    date    │ number_active_users │ percentage_active_users │
│    date    │        int64        │         double          │
├────────────┼─────────────────────┼─────────────────────────┤
│ 2019-01-01 │                  70 │                   34.65 │
│ 2019-01-02 │                  95 │                   47.03 │
│ 2019-01-03 │                 102 │                    50.5 │
│ 2019-01-04 │                 107 │                   52.97 │
│ 2019-01-05 │                 108 │                   53.47 │
│ 2019-01-06 │                 110 │                   54.46 │
│ 2019-01-07 │                 114 │                   56.44 │
│ 2019-01-08 │                 113 │                   55.94 │
│ 2019-01-09 │                 115 │                   56.93 │
│ 2019-01-10 │                 115 │                   56.93 │
│     ·      │                   · │                     ·   │
│     ·      │                   · │                   